# PDC Project

In [1]:
import numpy as np
from functools import reduce
import re

## 2)

In [3]:
def encode(string_of_bits):
    # String of bits to numpy array
    bit_array = np.array(list(map(int, string_of_bits)))
    # 2 bits per row
    bit_array_2d = bit_array.reshape(-1, 2)
    # Compute parity for each row
    parity_array = np.logical_xor(bit_array_2d[:,0], bit_array_2d[:,1]).astype(int)
    # Add parity column
    bit_array_with_parity_2d = np.block([bit_array_2d, parity_array.reshape(-1,1)])
    # Map [0,1] to [-1,1]
    return bit_array_with_parity_2d.flatten() * 2 - 1

In [4]:
def decode(encoded):
    encoded_2d = encoded.reshape(-1, 3)
    # Map [-1,1] to [0,1]
    bit_encoded_2d = (encoded_2d + 1) // 2

    # Loop over the erased bits
    for i, j in zip(*np.where(encoded_2d == 0)):
        # We can recover the bit by xoring the two other bits
        # xor <=> addition mod 2
        bit_encoded_2d[i][j] = np.sum(np.delete(bit_encoded_2d[i], j)) % 2

    # Remove parity and flatten
    decoded = bit_encoded_2d[:, :2].flatten()

    # Convert to string of bits
    return ''.join(decoded.astype(str))

In [5]:
def channel(chanInput):
    c = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3)
    c[erasedIndex:len(chanInput):3] = 0
    return c

In [6]:
def process(string):
    sender = str_to_bits(string)
    print(f"first we decompose our message in bits: {sender}")
    X = encode(sender)
    print(f"Encode it with the check bit and-1,1 : {X}")
    Y = channel(X)
    print(Y.shape)
    print(f"The channel erase some symbols : {Y}")
    estimator = decode(Y)
    print(f"Decode : {estimator}")
    original_message = bits_to_str(estimator)
    print(f"Convert bits to string : {message}")

In [7]:
process_modeling("Miaw")

first we decompose our message in bits: 01001101011010010110000101110111
Then we format it with the check bit and cast in -1,1 : [-1  1  1 -1 -1 -1  1  1 -1 -1  1  1 -1  1  1  1 -1  1  1 -1  1 -1  1  1
 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1 -1  1  1  1  1 -1 -1  1  1  1  1 -1]
(48,)
Then the channel erase some symbols : [ 0  1  1  0 -1 -1  0  1 -1  0  1  1  0  1  1  0 -1  1  0 -1  1  0  1  1
  0  1  1  0 -1  1  0 -1 -1  0  1  1  0  1  1  0  1 -1  0  1  1  0  1 -1]
We retrieve our initial bit string with the help of the check bits : 01001101011010010110000101110111
We inverse the bits into a string : Miaw
